In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_addons as tfa
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
sentence =("if you want to build a ship, dont't drum up people together tocollect wood and don't assign them tasks and work,"
         " but rather teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [21]:
seq_length = 10
dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i + seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, dont
23 ship, dont -> hip, dont'
24 hip, dont' -> ip, dont't
25 ip, dont't -> p, dont't 
26 p, dont't  -> , dont't d
27 , dont't d ->  dont't dr
28  dont't dr -> dont't dru
29 dont't dru -> ont't drum
30 ont't drum -> nt't drum 
31 nt't drum  -> t't drum u
32 t't drum u -> 't drum up
33 't drum up -> t drum up 
34 t drum up  ->  drum up p
35  drum up p -> drum up pe
36

In [5]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [6]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = seq_length
batch_size = len(dataX)

In [7]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

In [8]:
x_one_hot = tf.one_hot(X, num_classes)

In [9]:
cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)

C:\Users\since\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:699: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "


###### 다중 cell 쌓기
- MultiRNNcell에 cell * n 쌓고 싶은 층만큼 n를 곱해주면됨

In [10]:
cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([cell] * 2, state_is_tuple=True)

In [11]:
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot ,dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\since\anaconda3\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


# softmax를 활용하여 다중 RNN을 받기 

###### hidden_size - input // num_classes - output
- x_for_softmax가 data 값과 동일한 역할로 보면됨
- softmax_w의 경우 input을 받고 output으로 나감
- softmax_b의 경우 softmax_w의 output 값을 shape으로 받음 

In [12]:
x_for_softmax = tf.reshape(outputs, [-1, hidden_size])

In [13]:
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])

In [14]:
outputs = tf.matmul(x_for_softmax, softmax_w) + softmax_b

In [15]:
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

In [16]:
weights = tf.ones([batch_size, seq_length])

###### logits부분에 활성함수가 포함되지 않은 output 값이 들어가야함

In [17]:
sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)

In [19]:
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
for i in range(500):
    _, l, results = sess.run([train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    
    for j, result in enumerate(results):
        index = np.argmax(result, axis = 1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

0 0 mmskkkkkkk 3.2601864
0 1 mmmssskkss 3.2601864
0 2 mmssssssss 3.2601864
0 3 mmmsssssss 3.2601864
0 4 mmssssssss 3.2601864
0 5 mmssmsssss 3.2601864
0 6 mmmmmsssss 3.2601864
0 7 mmmmmsssss 3.2601864
0 8 mmmmssssss 3.2601864
0 9 mmmsssssss 3.2601864
0 10 msssssssss 3.2601864
0 11 msssssssss 3.2601864
0 12 mssssssmsm 3.2601864
0 13 mmmmmmmmmm 3.2601864
0 14 mmmmmmmmmm 3.2601864
0 15 mmmmmmmmmm 3.2601864
0 16 mmmmmmmmmk 3.2601864
0 17 mmmmmmmmmm 3.2601864
0 18 mmmmmmmmms 3.2601864
0 19 mmmmmmmmss 3.2601864
0 20 mmmmssssss 3.2601864
0 21 mmmmssssks 3.2601864
0 22 mmmssskkks 3.2601864
0 23 mmmmkkkkss 3.2601864
0 24 mmmsskssss 3.2601864
0 25 mmssssssss 3.2601864
0 26 mmmmsmsmss 3.2601864
0 27 mmmsmsmsss 3.2601864
0 28 mmmmmmmmmm 3.2601864
0 29 mmmmmmmmmm 3.2601864
0 30 mmmmmsmmmm 3.2601864
0 31 mmmmmmmmmm 3.2601864
0 32 mmmsmmmmms 3.2601864
0 33 mmmmmmmmmm 3.2601864
0 34 msmmmmmmms 3.2601864
0 35 mmmmmmmmss 3.2601864
0 36 mmmmmmmmmk 3.2601864
0 37 mmmmsskkkk 3.2601864
0 38 mmmmmsssss 3.2601

6 0 tttttttttt 2.9167757
6 1 ettttttttt 2.9167757
6 2 tttttttttt 2.9167757
6 3 tttttttttt 2.9167757
6 4 tttttttttt 2.9167757
6 5 tttttttttt 2.9167757
6 6 tttttttttt 2.9167757
6 7 ettttttttt 2.9167757
6 8 tttttttttt 2.9167757
6 9 tttttttttt 2.9167757
6 10 tttttttttt 2.9167757
6 11 tttttttttt 2.9167757
6 12 tttttttttt 2.9167757
6 13 tttttttttt 2.9167757
6 14 tttttttttt 2.9167757
6 15 tttttttttt 2.9167757
6 16 tttttttttt 2.9167757
6 17 tttttttttt 2.9167757
6 18 tttttttttt 2.9167757
6 19 tttttttttt 2.9167757
6 20 tttttttttt 2.9167757
6 21 tttttttttt 2.9167757
6 22 tttttttttt 2.9167757
6 23 tttttttttt 2.9167757
6 24 tttttttttt 2.9167757
6 25 tttttttttt 2.9167757
6 26 ettttttttt 2.9167757
6 27 ettttttttt 2.9167757
6 28 tttttttttt 2.9167757
6 29 tttttttttt 2.9167757
6 30 tttttttttt 2.9167757
6 31 tttttttttt 2.9167757
6 32 tttttttttt 2.9167757
6 33 tttttttttt 2.9167757
6 34 tttttttttt 2.9167757
6 35 tttttttttt 2.9167757
6 36 tttttttttt 2.9167757
6 37 tttttttttt 2.9167757
6 38 tttttttttt 2.9167

11 0            2.839128
11 1            2.839128
11 2            2.839128
11 3            2.839128
11 4            2.839128
11 5            2.839128
11 6            2.839128
11 7            2.839128
11 8            2.839128
11 9            2.839128
11 10            2.839128
11 11            2.839128
11 12            2.839128
11 13            2.839128
11 14            2.839128
11 15            2.839128
11 16            2.839128
11 17            2.839128
11 18            2.839128
11 19            2.839128
11 20            2.839128
11 21            2.839128
11 22            2.839128
11 23            2.839128
11 24            2.839128
11 25            2.839128
11 26            2.839128
11 27            2.839128
11 28            2.839128
11 29            2.839128
11 30            2.839128
11 31            2.839128
11 32            2.839128
11 33            2.839128
11 34            2.839128
11 35            2.839128
11 36            2.839128
11 37            2.839128
11 38            2.839

15 0            2.7880788
15 1            2.7880788
15 2            2.7880788
15 3            2.7880788
15 4            2.7880788
15 5            2.7880788
15 6            2.7880788
15 7            2.7880788
15 8            2.7880788
15 9            2.7880788
15 10            2.7880788
15 11            2.7880788
15 12            2.7880788
15 13            2.7880788
15 14            2.7880788
15 15            2.7880788
15 16            2.7880788
15 17            2.7880788
15 18            2.7880788
15 19            2.7880788
15 20            2.7880788
15 21            2.7880788
15 22            2.7880788
15 23            2.7880788
15 24            2.7880788
15 25            2.7880788
15 26            2.7880788
15 27            2.7880788
15 28            2.7880788
15 29            2.7880788
15 30            2.7880788
15 31            2.7880788
15 32            2.7880788
15 33            2.7880788
15 34            2.7880788
15 35            2.7880788
15 36            2.7880788
15 37      

19 0            2.6352592
19 1  o    o    2.6352592
19 2      o     2.6352592
19 3     o      2.6352592
19 4    o    oo 2.6352592
19 5   o    ooo 2.6352592
19 6  o    ooo  2.6352592
19 7 o    ooo   2.6352592
19 8    oooo    2.6352592
19 9    ooo     2.6352592
19 10  oooo      2.6352592
19 11  ooo     o 2.6352592
19 12  oo        2.6352592
19 13  o       o 2.6352592
19 14          o 2.6352592
19 15        oo  2.6352592
19 16       oo   2.6352592
19 17    o oo    2.6352592
19 18   oooo     2.6352592
19 19  oooo      2.6352592
19 20   oo       2.6352592
19 21   o      o 2.6352592
19 22  o    o o  2.6352592
19 23      o o   2.6352592
19 24     o o    2.6352592
19 25    o o     2.6352592
19 26   o o      2.6352592
19 27  o n       2.6352592
19 28  oo        2.6352592
19 29  o         2.6352592
19 30            2.6352592
19 31          o 2.6352592
19 32    o    o  2.6352592
19 33   o    o   2.6352592
19 34  oo   o    2.6352592
19 35  o   o     2.6352592
19 36     o      2.6352592
19 37      

...

482 0 g you want 0.2287978
482 1  you want  0.2287978
482 2 tou want t 0.2287978
482 3  u want to 0.2287978
482 4 n want to  0.2287978
482 5 twant to b 0.2287978
482 6 tont to bu 0.2287978
482 7 ont to bui 0.2287978
482 8 nd to buil 0.2287978
482 9 d to build 0.2287978
482 10  wo build  0.2287978
482 11 to luild a 0.2287978
482 12   luild a  0.2287978
482 13 nluild a s 0.2287978
482 14 tutld a sh 0.2287978
482 15 utld a shi 0.2287978
482 16 tld a ship 0.2287978
482 17 gd a ship, 0.2287978
482 18 e a ship,  0.2287978
482 19  a ship, d 0.2287978
482 20 tnship, do 0.2287978
482 21 nship, don 0.2287978
482 22 thip, dont 0.2287978
482 23 sip, dont' 0.2287978
482 24 ep, dont't 0.2287978
482 25 g, dont't  0.2287978
482 26 , dont't d 0.2287978
482 27  dont't dr 0.2287978
482 28 ton''t dru 0.2287978
482 29  nt't drum 0.2287978
482 30 nt't drum  0.2287978
482 31 d t drum u 0.2287978
482 32  t drum up 0.2287978
482 33 t arum up  0.2287978
482 34  wrum up p 0.2287978
482 35 toum up pe 0.2287978
48

485 10 hao build  0.22878274
485 11 to luild a 0.22878274
485 12 h luild a  0.22878274
485 13 nluild a s 0.22878274
485 14 tuild a sh 0.22878274
485 15 uild a shi 0.22878274
485 16 pld a ship 0.22878274
485 17 md a ship, 0.22878274
485 18 e a ship,  0.22878274
485 19  a ship, d 0.22878274
485 20 tnship, do 0.22878274
485 21 nship, don 0.22878274
485 22 thip, dont 0.22878274
485 23  ip, dont' 0.22878274
485 24 ep, dont't 0.22878274
485 25 m, dont't  0.22878274
485 26 e dont't d 0.22878274
485 27  bont't dr 0.22878274
485 28 tont't dru 0.22878274
485 29  n''t drum 0.22878274
485 30 ng't drum  0.22878274
485 31 d't drum u 0.22878274
485 32 ht drum up 0.22878274
485 33 t drum up  0.22878274
485 34 harum up p 0.22878274
485 35 toum up pe 0.22878274
485 36  um up peo 0.22878274
485 37  m up peop 0.22878274
485 38 p up peopl 0.22878274
485 39  tp people 0.22878274
485 40 tp people  0.22878274
485 41 p people t 0.22878274
485 42 epeople to 0.22878274
485 43 teople tog 0.22878274
485 44 eople t

489 0 p you want 0.22879077
489 1  you want  0.22879077
489 2 tou want t 0.22879077
489 3  u want to 0.22879077
489 4 n want to  0.22879077
489 5 mwant to b 0.22879077
489 6 tont to bu 0.22879077
489 7 ont to bui 0.22879077
489 8 nd to buil 0.22879077
489 9 d'to build 0.22879077
489 10  ao build  0.22879077
489 11 to luild a 0.22879077
489 12   luild a  0.22879077
489 13 nluild a s 0.22879077
489 14 tutld a sh 0.22879077
489 15 utld a shi 0.22879077
489 16 mld a ship 0.22879077
489 17 pd a ship, 0.22879077
489 18 e a ship,  0.22879077
489 19  a ship, d 0.22879077
489 20 tnship, do 0.22879077
489 21 nship, don 0.22879077
489 22 thip, dont 0.22879077
489 23 iip, dont' 0.22879077
489 24 ep, dont't 0.22879077
489 25 p, dont't  0.22879077
489 26 , dont't d 0.22879077
489 27  dont't dr 0.22879077
489 28 ton''t dru 0.22879077
489 29  n''t drum 0.22879077
489 30 n''t drum  0.22879077
489 31 d't drum u 0.22879077
489 32  t drum up 0.22879077
489 33 t arum up  0.22879077
489 34  arum up p 0.2287

493 0 p you want 0.22892927
493 1 oyou want  0.22892927
493 2 tou want t 0.22892927
493 3  u want to 0.22892927
493 4 n want to  0.22892927
493 5 pwant to b 0.22892927
493 6 tont to bu 0.22892927
493 7 ont to bui 0.22892927
493 8 nd to buil 0.22892927
493 9 d'to build 0.22892927
493 10  ao build  0.22892927
493 11 to luild a 0.22892927
493 12   luild a  0.22892927
493 13 nluild a s 0.22892927
493 14 tutld a sh 0.22892927
493 15 utld a shi 0.22892927
493 16 pld a ship 0.22892927
493 17 pd a ship, 0.22892927
493 18 e a ship,  0.22892927
493 19  anship, d 0.22892927
493 20 tnship, do 0.22892927
493 21 nship, don 0.22892927
493 22 thip, dont 0.22892927
493 23 sip, dont' 0.22892927
493 24 ep, dont't 0.22892927
493 25 p, dont't  0.22892927
493 26 , dont't d 0.22892927
493 27  dont't dr 0.22892927
493 28 ton''t dru 0.22892927
493 29  n''t drum 0.22892927
493 30 ng't drum  0.22892927
493 31 d't drum u 0.22892927
493 32  t drum up 0.22892927
493 33 t arum up  0.22892927
493 34  arum up p 0.2289

496 43 teople tog 0.22981846
496 44 eople toge 0.22981846
496 45  ple toget 0.22981846
496 46 nle togeth 0.22981846
496 47 ee togethe 0.22981846
496 48 ectogether 0.22981846
496 49  together  0.22981846
496 50 to ether t 0.22981846
496 51 h ether to 0.22981846
496 52 nether toc 0.22981846
496 53  ther toco 0.22981846
496 54  her tocol 0.22981846
496 55 her tocoll 0.22981846
496 56 er tocolle 0.22981846
496 57   tocollec 0.22981846
496 58  thcollect 0.22981846
496 59 to ollect  0.22981846
496 60 h ollect w 0.22981846
496 61 nollect wo 0.22981846
496 62 hllect woo 0.22981846
496 63 nlect wood 0.22981846
496 64 eect wood  0.22981846
496 65 ect wood a 0.22981846
496 66  t wood an 0.22981846
496 67 h wood and 0.22981846
496 68 htood and  0.22981846
496 69 tord and d 0.22981846
496 70 ord and do 0.22981846
496 71 nd and don 0.22981846
496 72 n and don' 0.22981846
496 73  a d don't 0.22981846
496 74 tnd won't  0.22981846
496 75 nd won't a 0.22981846
496 76 d won't as 0.22981846
496 77  aon't 

In [24]:
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis = 1)
    if j is 0:
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

g you want to build a ship, dont't drum up people together tocollect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\since\AppData\Local\Temp/ipykernel_15968/2500914604.py:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if j is 0:
